In [ ]:
logging = False
submit = True
supplemental = True

if logging:
    !pip install neptune-client
    !pip install neptune-client[lightgbm]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
import jpx_tokyo_market_prediction
from lightgbm import LGBMRegressor
from lightgbm import log_evaluation
from sklearn.linear_model import Ridge
import math 

import seaborn as sns
pd.set_option('display.max_rows', 999)
pd.options.mode.chained_assignment = None  # default='warn'
from decimal import ROUND_HALF_UP, Decimal

from sklearn.metrics import mean_squared_error

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
sub_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/sample_submission.csv")
options_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/options.csv")
fin_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/financials.csv")
sec_sprice_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/secondary_stock_prices.csv")
trades_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/trades.csv")
sprice_ex_test = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/example_test_files/stock_prices.csv")

In [ ]:
options_train = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/options.csv")
fin_train = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv")
sec_sprice_train = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/secondary_stock_prices.csv")
trades_train = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/trades.csv")
sprice_train = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv")

In [ ]:
options_supp = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/options.csv")
fin_supp = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/financials.csv")
sec_sprice_supp = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/secondary_stock_prices.csv")
trades_supp = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/trades.csv")
sprice_supp = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
stock_list = pd.read_csv(r"/kaggle/input/jpx-tokyo-stock-exchange-prediction/stock_list.csv")

In [ ]:
if supplemental:
    sprice_train = pd.concat([sprice_train, sprice_supp], axis = 0)

# Prepare data

In [ ]:
#official adjust close price calc, https://www.kaggle.com/code/smeitoma/train-demo/notebook

def adjust_price(price):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
    Returns:
        price DataFrame (pd.DataFrame): stock_price with generated AdjustedClose
    """
    # transform Date column into datetime
    price.loc[: ,"Date"] = pd.to_datetime(price.loc[: ,"Date"], format="%Y-%m-%d")

    def generate_adjusted_close(df):
        """
        Args:
            df (pd.DataFrame)  : stock_price for a single SecuritiesCode
        Returns:
            df (pd.DataFrame): stock_price with AdjustedClose for a single SecuritiesCode
        """
        # sort data to generate CumulativeAdjustmentFactor
        df = df.sort_values("Date", ascending=False)
        # generate CumulativeAdjustmentFactor
        df.loc[:, "CumulativeAdjustmentFactor"] = df["AdjustmentFactor"].cumprod()
        # generate AdjustedClose
        df.loc[:, "AdjustedClose"] = (
            df["CumulativeAdjustmentFactor"] * df["Close"]
        ).map(lambda x: float(
            Decimal(str(x)).quantize(Decimal('0.1'), rounding=ROUND_HALF_UP)
        ))
        # reverse order
        df = df.sort_values("Date")
        # to fill AdjustedClose, replace 0 into np.nan
        df.loc[df["AdjustedClose"] == 0, "AdjustedClose"] = np.nan
        # forward fill AdjustedClose
        df.loc[:, "AdjustedClose"] = df.loc[:, "AdjustedClose"].ffill()
        return df

    # generate AdjustedClose
    price = price.sort_values(["SecuritiesCode", "Date"])
    price = price.groupby("SecuritiesCode").apply(generate_adjusted_close).reset_index(drop=True)

    price.set_index("Date", inplace=True)
    return price

In [ ]:
df_price_raw = sprice_train
prices_train = adjust_price(df_price_raw)

In [ ]:
prices_train

In [ ]:
def get_RSI(df, close_col):
    
    df = df.copy()
    df =  df.sort_index(ascending = True)
    
    df["Close_diff"] = df[close_col].diff(1)
    
    df["gain"] = [diff if diff >= 0 else 0 for diff in df["Close_diff"]]
    df["loss"] = [diff*-1 if diff < 0 else 0 for diff in df["Close_diff"]]
    
    df["avg_gain"] = df["gain"].rolling(window = 14).mean()
    df["avg_loss"] = df["loss"].rolling(window = 14).mean()
    
    df["RS"] = df["avg_gain"] / df["avg_loss"]
    df["RSI"] = 100 - (100/(1+df["RS"]))
    
    return df[["RSI"]]

In [ ]:
stock_list["MarketCapitalization"] = np.log1p(stock_list["MarketCapitalization"])
stock_list["company_size_proxy"] = pd.cut(stock_list["MarketCapitalization"], bins = 4, labels = list(range(1,5)))
stock_list["issued_shares_cat"] = pd.cut(np.log1p(stock_list["IssuedShares"]), bins = 3, labels = list(range(1,4)))
stock_list["NewIndexSeriesSizeCode"] = [int(row) if row!="-" else 999 for row in stock_list["NewIndexSeriesSizeCode"]]

In [ ]:
#inspired from: https://www.kaggle.com/code/smeitoma/train-demo/notebook

def get_features_for_predict(price, code):
    """
    Args:
        price (pd.DataFrame)  : pd.DataFrame include stock_price
        code (int)  : A local code for a listed company
    Returns:
        feature DataFrame (pd.DataFrame)
    """
    close_col = "AdjustedClose"
    feats = price.loc[price["SecuritiesCode"] == code, ["SecuritiesCode", close_col]].copy()
    
    periods = [10, 21, 63]
    
    for period in periods:
        feats.loc[:, f"return_{period}"] = feats["AdjustedClose"].pct_change(period)
        feats.loc[:, f"volatility_{period}"] = np.log(feats["AdjustedClose"]).diff().rolling(period).std()
    
    feats.loc[:, "RSI"] = get_RSI(feats, close_col)
    
    #bollinger bands
    feats.loc[:, "rollingMean20"] = feats["AdjustedClose"].rolling(window = 20).mean()
    feats.loc[:, "upperBollinger20"] = feats["rollingMean20"] + 2 * feats["AdjustedClose"].rolling(window = 20).std()
    feats.loc[:, "lowerBollinger20"] = feats["rollingMean20"] - 2 * feats["AdjustedClose"].rolling(window = 20).std()
    
    #time features
    feats.loc[:, "day_of_week"] = feats.index.dayofweek
    feats.loc[:, "month"] = feats.index.month
    
    feats["rng_help"] = feats.index.dayofyear
    feats["rng_help_norm"] = 2 * math.pi * feats["rng_help"] /feats["rng_help"].max()
    feats["cos_x"] = np.cos(feats["rng_help_norm"])
    feats["sin_x"] = np.sin(feats["rng_help_norm"])
    feats.drop(["rng_help", "rng_help_norm"], axis = 1, inplace = True)
        
    
    #quantiles 
    feats["quantile_75"] = feats["AdjustedClose"].rolling(window = 20).quantile(quantile = 0.75)
    feats["quantile_25"] = feats["AdjustedClose"].rolling(window = 20).quantile(quantile = 0.25)
    feats["IQR"] = feats["quantile_75"] - feats["quantile_25"]
    
    # filling data for nan and inf
    feats = feats.fillna(0)
    feats = feats.replace([np.inf, -np.inf], 0)
    
    #add company size proxy, sector, issued shares category, NewMarketSegment
    feats = feats.reset_index()
    feats = pd.merge(left = feats, right = stock_list[["SecuritiesCode", "company_size_proxy", "issued_shares_cat", "17SectorCode","NewIndexSeriesSizeCode"]], left_on = "SecuritiesCode", right_on = "SecuritiesCode", how = "left")
    feats.set_index("Date", inplace = True)
    
    # drop AdjustedClose column
    feats = feats.drop([close_col], axis=1)

    return feats

In [ ]:
# fetch prediction target SecuritiesCodes
codes = sorted(prices_train["SecuritiesCode"].unique())
len(codes)

In [ ]:
# generate feature
from tqdm import tqdm

buff = []
for code in tqdm(codes):
    feat = get_features_for_predict(prices_train, code)
    buff.append(feat)
feature = pd.concat(buff)
rows_before = feature.shape[0]
feature.dropna(inplace = True)
rows_after = feature.shape[0]

print(f"{rows_after - rows_before} have been dropped")

In [ ]:
def reduce_mem_usage(df):
    # iterate through all the columns of a dataframe and modify the data type
    #   to reduce memory usage. Credits to Guillaume Martin
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
features = reduce_mem_usage(feature)

In [ ]:
target_col = prices_train[["SecuritiesCode", "Target"]]
merged_train_data = pd.merge(left = feature, right = target_col, left_on = [feature.index, "SecuritiesCode"], right_on = [target_col.index, "SecuritiesCode"]).set_index("key_0", drop = True)
merged_train_data.index.names = ["Date"]
merged_train_data.dropna(inplace = True)

merged_train_data

# Train and choose model

### functions

In [ ]:
def set_rank(df):
    """
    Args:
        df (pd.DataFrame): including predict column
    Returns:
        df (pd.DataFrame): df with Rank
    """
    df =  df.sort_values("target_pred", ascending = False)
    df.loc[:, "Rank"] = np.arange(len(df["target_pred"]))
    return df


#evaluation function to calculate the sharp ratio

def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

### Setup neptune project

In [ ]:
merged_train_data.head().to_csv("./merged_train_data_sample.csv")

In [ ]:
#model definition and tracking

params ={
    "lgb_params" : {
        'n_estimators': 500,
        'num_leaves' : 100,
        'learning_rate': 0.1,
        'colsample_bytree': 0.9,
        'subsample': 0.8,
        'reg_alpha': 0.4,
        'metric': 'mae',
        'random_state': 21
    }
}

cols = merged_train_data.columns

if logging:     
    
    import neptune.new as neptune
    from neptune.new.integrations.lightgbm import NeptuneCallback
    from kaggle_secrets import UserSecretsClient

    user_secrets = UserSecretsClient()
    api_token = user_secrets.get_secret("neptune_token")

    run = neptune.init(project='rb.wesselmann/JPX-stock-kaggle-Project', api_token = api_token)
    neptune_callback = NeptuneCallback(run=run)
    
    run["name"] = "baseline_v10_inkl_suppl"
    run["algorithm"] = "LightGBM" 
    run["parameters"] = params
    run["features"] = list(cols)
    run["dataset/merged_train_data_sample"].upload("./merged_train_data_sample.csv")

### Use Time Series CV to choose the model

In [ ]:
feature_cols = merged_train_data.drop("Target", axis = 1).columns

if not submit:

    from sklearn.model_selection import TimeSeriesSplit

    tscv = TimeSeriesSplit(n_splits = 5)
    score = {}

    print("beginning CV ...\n")

    for fold, (train_index, val_index) in enumerate(tscv.split(merged_train_data)):

        print(f"------------- Fold {fold} -------------")
        print(f"Train index: from {merged_train_data.iloc[train_index].index[0]} to {merged_train_data.iloc[train_index].index[-1]}")
        print(f"Validation index: from {merged_train_data.iloc[val_index].index[0]} to {merged_train_data.iloc[val_index].index[-1]}")

        X_train, y_train = merged_train_data[feature_cols].iloc[train_index], merged_train_data["Target"].iloc[train_index]
        X_val, y_val = merged_train_data[feature_cols].iloc[val_index], merged_train_data["Target"].iloc[val_index]

        #train
        lgb_model = LGBMRegressor(**params["lgb_params"])
        lgb_model.fit(X_train[feature_cols], y_train, eval_set = (X_val, y_val), early_stopping_rounds = 100, callbacks = [neptune_callback])
        lgb_model.fit(X_train[feature_cols], y_train)
                  
        #predict
        result = X_val[["SecuritiesCode"]].copy()
        result.loc[:, "target_pred"] = 1 * lgb_model.predict(X_val[feature_cols])
        result.loc[:, "Target"] = y_val

        #rank
        result = result.sort_values(["Date", "target_pred"], ascending = [True, False])
        result = result.groupby("Date").apply(set_rank)

        score[f"Fold_{fold}"] = calc_spread_return_sharpe(result, portfolio_size = 200)
        print(f"Sharp Ratio for Fold {fold}: ", score[f"Fold_{fold}"], "\n")

        if logging:
            run["metrics/Sharp_ratio"].log(score[f"Fold_{fold}"])

    print(f"Average sharp ratio: {sum(score.values())/len(score)}")

    if logging:
        run["avg_Sharp_ratio"] = sum(score.values())/len(score)

In [ ]:
if not submit:
    importance = pd.DataFrame({"columns":X_train.columns, "importance" : lgb_model.feature_importances_})
    importance = importance.sort_values(by = "importance", ascending = False)

    fig, sub = plt.subplots(1,1,figsize=(15,8))
    sns.barplot(y = importance["columns"], x = importance["importance"], orient = "hor", ax = sub)
    sub.grid()

In [ ]:
if logging:
    import joblib

    run["model_lgb"].upload(joblib.dump(lgb_model, "lgb_model.pkl")[0])
    run.stop()

In [ ]:
X, y = merged_train_data[feature_cols], merged_train_data["Target"]

lgb_model = LGBMRegressor(**params["lgb_params"])
lgb_model.fit(X[feature_cols], y)

In [ ]:
X.columns

In [ ]:
set(feature_cols) == set(X.columns)

# Submission baseline

In [ ]:
# load Time Series API
import jpx_tokyo_market_prediction
# make Time Series API environment (this function can be called only once in a session)
env = jpx_tokyo_market_prediction.make_env()
# get iterator to fetch data day by day
iter_test = env.iter_test()

In [ ]:
price_cols = [
    "Date",
    "SecuritiesCode",
    "Close",
    "AdjustmentFactor"
]

df_price_raw = df_price_raw[price_cols]

counter = 0

#fetch data day by day
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    
    current_date = prices["Date"].iloc[0]
    sample_prediction_date = sample_prediction["Date"].iloc[0]
    print(f"current date: {current_date}, sample_prediction_date: {sample_prediction_date}")
    
    if counter == 0: 
        
        df_price_raw = df_price_raw.loc[df_price_raw["Date"] < current_date]
        
    threshold = (pd.Timestamp(current_date) - pd.offsets.BDay(80)).strftime("%Y-%m-%d")
    print(f"threshold: {threshold}")
    df_price_raw = df_price_raw.loc[df_price_raw["Date"] >= threshold]
    
    df_price_raw = pd.concat([df_price_raw, prices[price_cols]])
    df_price = adjust_price(df_price_raw)
    
    codes = sorted(prices["SecuritiesCode"].unique())
    
    #predict
    feature = pd.concat([get_features_for_predict(df_price, code) for code in codes])
    feature = feature.loc[feature.index == current_date]
    feature = reduce_mem_usage(feature)
    
    feature.loc[:, "predict"] = lgb_model.predict(feature[feature_cols]) 
    
    #set rank
    feature = feature.sort_values("predict", ascending = False).drop_duplicates(subset = ["SecuritiesCode"])
    feature.loc[:, "Rank"] = np.arange(len(feature))
    feature_map = feature.set_index("SecuritiesCode")["Rank"].to_dict()
    sample_prediction["Rank"] = sample_prediction["SecuritiesCode"].map(feature_map)
    
    #chk rank
    assert sample_prediction["Rank"].notna().all()
    assert sample_prediction["Rank"].min() == 0
    assert sample_prediction["Rank"].max() == len(sample_prediction["Rank"]) - 1
    
    env.predict(sample_prediction)
    counter += 1        